In [85]:
%run ../00_AdvancedPythonConcepts/talktools.py

# Unsupervised Learning

Not all historical "answers" known in inference problems. Sometimes we look for structure without the ability to score: clustering. Sometimes we want to find lower-dimensional embeddings of large-dimensional data.

## Clustering

<img src="https://www.evernote.com/l/AUWSPWZPSkNAL6GLEX37sIbG7gYbvSUyR6UB/image.png">

### KMeans

In [ ]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn import datasets
dataset = datasets.load_iris()
X = dataset.data
y = dataset.target

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X)
labels = kmeans_model.labels_

In [ ]:
list(zip(labels, y))

From [sklearn](http://scikit-learn.org/stable/modules/clustering.html):

*"Evaluating the performance of a clustering algorithm is not as trivial as counting the number of errors or the precision and recall of a supervised classification algorithm. In particular any evaluation metric should not take the absolute values of the cluster labels into account but rather if this clustering define separations of the data similar to some ground truth set of classes or satisfying some assumption such that members belong to the same class are more similar that members of different classes according to some similarity metric."*

*Fowlkes-Mallows* can be used when the ground truth class assignments of the samples is known.

In [ ]:
metrics.fowlkes_mallows_score(y, labels)

If the ground truth labels are not known, evaluation must be performed using the model itself.   *Silhouette Coefficient*: a higher score relates to a model with better defined clusters.

In [ ]:
metrics.silhouette_score(X, labels, metric='euclidean')

### DBSCAN

"DBSCAN - Density-Based Spatial Clustering of Applications with Noise. Finds core samples of high density and expands clusters from them. Good for data which contains clusters of similar density...This implementation is by default not memory efficient because it constructs a full pairwise similarity matrix in the case where kd-trees or ball-trees cannot be used (e.g. with sparse matrices). This matrix will consume <b>n<sup>2</sup></b> floats."

http://scikit-learn.org/stable/modules/clustering.html#dbscan

In [ ]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler

In [ ]:
centers = [[1, 1, 0,1], [-1, -1, 2,-1.5], [1, -1, -1,1]]
X, labels_true = make_blobs(n_samples=1000, centers=centers, cluster_std=0.5,
                            random_state=42)

X = StandardScaler().fit_transform(X)
xs = X[:,0] ; ys = X[:,1] ; zs = X[:,2]

In [ ]:
X.shape

In [ ]:
set(labels_true)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs, ys, zs=zs, c=labels_true)

In [ ]:
db = DBSCAN(eps=0.4, min_samples=10,algorithm="kd_tree",n_jobs=-1).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

In [ ]:
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = 'k'

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=6,alpha=0.2)

plt.title('Estimated number of clusters: %d' % n_clusters_)

# Dimensionality Reduction

In our toy example above, we have a 4-D distribution over 3 clusters. Let's say we wish to find a lower-D space to transform our data. We can use Principal Component Analysis, to identify the (linear) combination of attributes that account for most of the variance in the data.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3, svd_solver='full')
pca.fit(X)
print(pca.explained_variance_ratio_)

In [ ]:
X_r = pca.transform(X)

In [ ]:
X_r

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_r[:,0],X_r[:,1],X_r[:,2], c=labels_true)

"Linear Discriminant Analysis (LDA) tries to identify attributes that account for the most variance between classes. In particular, LDA, in contrast to PCA, is a supervised method, using known class labels."

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=2)
lda.fit(X,labels_true)
print(lda.explained_variance_ratio_) 

In [ ]:
X_r = lda.transform(X)
plt.scatter(X_r[:,0], X_r[:,1], c=labels_true)

In [ ]:
X_r.shape

In [ ]:
from sklearn import datasets

digits = datasets.load_digits()

X = digits.data
y = digits.target

In [ ]:
y[0]

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=3)
lda.fit(X,y)
print(lda.explained_variance_ratio_) 

In [ ]:
X_r = lda.transform(X)

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

cax=ax.scatter(X_r[:,0],X_r[:,1],X_r[:,2],c=y)
cbar = fig.colorbar(cax)

## Manifold Learning

non-linear dimensionality reduction

In [ ]:
%matplotlib notebook 
from sklearn import manifold
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets

n_points = 3000
X, t = datasets.samples_generator.make_swiss_roll(n_samples=n_points)
fig = plt.figure(figsize=(8,12))
ax = fig.add_subplot(211, projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], c=t)

ax1 = fig.add_subplot(212)
ax1.scatter(X[:,0], X[:,2], c=t)

In [ ]:
manifold.LocallyLinearEmbedding?

In [ ]:
## Locally Linear Embedding .... like PCA on the local scale (preserving neighborhood distances).

from sklearn import manifold
out_dim = 2
n_neighbors = 20

Y = manifold.LocallyLinearEmbedding(n_neighbors, out_dim, eigen_solver='auto',\
                                    method='standard').fit_transform(X)
plt.figure(figsize=(10, 8))
plt.scatter(Y[:, 0], Y[:, 1], c=t)
plt.title("LLE",size=25)

In [ ]:
manifold.Isomap?

In [ ]:
# IsoMap is a non-linear kernel PCA
Y = manifold.Isomap(n_neighbors, out_dim).fit_transform(X)

plt.figure(figsize=(8, 8))
plt.scatter(Y[:, 0], Y[:, 1], c=t)
plt.title("IsoMap",size=25)

## t-SNE

t-distributed Stochastic Neighbor Embedding.

t-SNE [1] is a tool to visualize high-dimensional data. It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results.

<i>"It is highly recommended to use another dimensionality reduction method (e.g. PCA for dense data or TruncatedSVD for sparse data) to reduce the number of dimensions to a reasonable amount (e.g. 50) if the number of features is very high."</i>

http://scikit-learn.org/stable/modules/manifold.html#t-sne


In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()

In [ ]:
X = iris.data
y = iris.target

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', random_state=0, perplexity=20, verbose=1)
X_tsne = tsne.fit_transform(X)

In [ ]:
%matplotlib inline
plt.scatter(X_tsne[:,0],X_tsne[:,1],c=y)
plt.title("t-SNE",size=25)


## umap

Uniform Manifold Approximation and Projection

"model the manifold with a fuzzy topological structure. The embedding is found by searching for a low dimensional projection of the data that has the closest possible equivalent fuzzy topological structure."

McInnes, L, Healy, J, UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction, ArXiv e-prints 1802.03426, 2018

https://arxiv.org/abs/1802.03426

In [ ]:
#!pip install umap-learn

In [ ]:
import umap
from sklearn.datasets import load_digits

digits = load_digits()

embedding = umap.UMAP(n_neighbors=5,
                      min_dist=0.3,
                      metric='correlation').fit_transform(digits.data)

In [ ]:
from bokeh.plotting import figure, output_file
from bokeh.models import CategoricalColorMapper, ColumnDataSource
from bokeh.palettes import Category10
from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
targets = [str(d) for d in digits.target_names]

source = ColumnDataSource(dict(
    x = [e[0] for e in embedding],
    y = [e[1] for e in embedding],
    label = [targets[d] for d in digits.target]
))

cmap = CategoricalColorMapper(factors=targets, palette=Category10[10])

p = figure(title="test umap")
p.circle(x='x',
         y='y',
         source=source,
         color={"field": 'label', "transform": cmap},
         legend='label')

show(p)